In [1]:
from ultralytics import YOLO
import torch
import os
import cv2
import numpy as np
import random

In [2]:
torch.cuda.is_available()

True

In [3]:
seed_value=446
random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)

In [4]:
torch.cuda.empty_cache()

In [6]:
# --- 設定路徑 ---
# 假設 data.yaml 已經準備好，並且指嚮您的圖片和標籤
DATA_YAML_PATH = 'data.yaml'

# --- 設定超參數 ---
# 針對移動端應用，使用非正方形輸入尺寸 (例如 16:9 比例)
IMG_WIDTH = 960
IMG_HEIGHT = 720
IMG_SIZE = (IMG_WIDTH, IMG_HEIGHT) 

PROJECT_FOLDER = "D:\大學資料\電腦視覺\期末專案\\dataset\\test2" # 專案根目錄
RUN_NAME = 'pre_test2'      # 本次訓練的名稱

MODEL_TYPE = 'yolov8s.pt'  # 使用 Nano 模型
EPOCHS = 1
BATCH_SIZE = 16

# --- 1. 載入模型 ---
model = YOLO(MODEL_TYPE)
print(f"--- 載入模型: {MODEL_TYPE}，輸入尺寸: {IMG_SIZE} ---")

# --- 2. 執行訓練 (資料同步導入與擴增都在這一行中自動完成) ---
# YOLO 框架會根據 data.yaml 找到圖片和標籤，
# 並在訓練時自動執行 Mosaic、縮放、翻轉等資料擴增，
# 同步更新偵測框座標。

print("正在開始訓練，數據導入與處理將由 ultralytics 自動管理...")
try:
    results = model.train(
        data=DATA_YAML_PATH,     # 資料集配置文件
        epochs=EPOCHS,           # 訓練輪數
        imgsz=IMG_SIZE,          # 圖片輸入尺寸 (非正方形)
        batch=BATCH_SIZE,        # 批次大小
        # 啟用/調整關鍵的資料擴增參數
        mosaic=1.0,              # Mosaic 擴增機率 (解決單物體問題)
        fliplr=0.5,              # 水平翻轉機率
        amp=True,
        project=PROJECT_FOLDER,
        name=RUN_NAME
    )
    print("訓練完成！")
except Exception as e:
    print(f"訓練過程中發生錯誤: {e}")
    print("請檢查 data.yaml 檔案路徑和格式是否正確。")

--- 載入模型: yolov8s.pt，輸入尺寸: (960, 720) ---
正在開始訓練，數據導入與處理將由 ultralytics 自動管理...
Ultralytics 8.3.235  Python-3.12.10 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=(960, 720), int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=pre_test22, nbs=64, nms=False, opset=None, optimize=Fal

In [1]:
MODEL_PATH = "D:\大學資料\電腦視覺\期末專案\\dataset\\test2\\pre_test1\\weights\\last.pt"

# 2. 待檢測圖片路徑：這裡使用單一圖片作為範例
#    (如果使用資料夾，程式會依序顯示資料夾內所有圖片)
SOURCE_PATH = "D:\大學資料\電腦視覺\期末專案\測試圖片2.png"

# --- 1. 載入訓練好的模型 ---
try:
    model = YOLO(MODEL_PATH)
    print(f"--- 成功載入模型權重: {MODEL_PATH} ---")
except Exception as e:
    print(f"錯誤：無法載入模型。請檢查路徑是否正確。錯誤訊息: {e}")
    exit()

# --- 2. 執行推論 (檢測) ---
print(f"正在對 {SOURCE_PATH} 執行物件偵測並即時顯示結果...")

# 執行 predict 並獲取 Results 物件列表
results = model.predict(
    source=SOURCE_PATH,        # 待檢測的圖片或資料夾                
    #conf=0.5,                  # 置信度閾值
    imgsz=(960,720),               # 檢測時的輸入圖片尺寸
    # --- 關鍵修改點 ---
    save=False,                # 設置為 False，不儲存結果圖片到硬碟
    show=True,                 # 設置為 True，直接彈出視窗顯示圖片
    # 設置輸出結果的儲存路徑為 None，確保不產生任何檔案
    save_txt=False,
    save_conf=False,
    save_crop=False,
    project=None,
    name=None
)

print("\\n--- 推論完成 ---")
print("如果使用的是單一圖片，視窗將會關閉。")
print("如果使用的是資料夾，將會依序顯示圖片。")

# --- 3. 程式碼控制顯示 (適用於終端機/單張圖片) ---
# 當 `show=True` 時，YOLO 內部會使用 OpenCV 顯示圖片。
# 如果是在單獨的 Python 環境運行，可能需要手動控制 OpenCV 視窗。

for result in results:
    # 提取帶有邊界框的圖片 (np.ndarray)
    im_bgr = result.plot()
    
    # 轉換顏色空間 (YOLO 內部使用 BGR，Matplotlib/OpenCV 顯示需要調整)
    # im_rgb = cv2.cvtColor(im_bgr, cv2.COLOR_BGR2RGB)
    
    # 使用 OpenCV 顯示視窗
    cv2.imshow("YOLOv8 Detection Result", im_bgr)
    cv2.waitKey(0) # 等待按鍵
    cv2.destroyAllWindows()

錯誤：無法載入模型。請檢查路徑是否正確。錯誤訊息: name 'YOLO' is not defined
正在對 D:\大學資料\電腦視覺\期末專案\測試圖片2.png 執行物件偵測並即時顯示結果...


NameError: name 'model' is not defined

: 